In [1]:
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import tensortools as tt
%matplotlib inline

In [2]:
from convokit import Corpus, download, Utterance, Speaker

In [ ]:
corpus = Corpus(download('subreddit-news'))

Dataset already exists at /Users/calebchiam/.convokit/downloads/subreddit-news


In [ ]:
corpus = corpus.filter_conversations_by(selector=lambda convo: 
                                         1483228800 <= convo.get_utterance(convo.id).timestamp <= 1514764800)

In [ ]:
top_level_comment_ids = [utt.id for utt in corpus.iter_utterances() if utt.id == utt.meta['top_level_comment']]
corpus = corpus.reindex_conversations(top_level_comment_ids)

In [ ]:
corpus.filter_utterances_b

In [10]:
roots = set(utt.conversation_id for utt in corpus.iter_utterances())
generic_utts = [Utterance(id=root, conversation_id=root, speaker=Speaker(id=root)) for root in roots]

In [11]:
corpus.add_utterances(generic_utts, with_checks=False)
corpus = corpus.reindex_conversations(top_level_comment_ids)

In [12]:
corpus = corpus.filter_conversations_by(selector=lambda convo: 
                                         convo.get_utterance(convo.id).meta['subreddit'] == 'news')

### Unidimensional TCA/SVD for utt length

In [17]:
import random
longest_paths = dict()
for convo in corpus.iter_conversations():
    longest_paths[convo.id] = random.choice(convo.get_longest_paths())
    convo.meta['longest_path_length'] = len(longest_paths[convo.id])

In [18]:
filtered_utts = []
for convo in corpus.iter_conversations(lambda convo: convo.meta['longest_path_length'] >= 10):
    filtered_utts.extend(longest_paths[convo.id])

In [19]:
len(filtered_utts)

3615

In [21]:
len(list(corpus.iter_conversations(lambda convo: convo.meta['longest_path_length'] >= 10)))

275

In [16]:
corpus.random_conversation().print_conversation_structure()

Allbanned1984
    Zillaplus2
        USER9675476
            nine_second_fart
    Chucknastical
        Allbanned1984
            Chucknastical
                Allbanned1984
            Gdfi
    Gdfi


In [13]:
corpus.print_summary_stats()

Number of Speakers: 11829
Number of Utterances: 24545
Number of Conversations: 1000
